<a href="https://colab.research.google.com/github/narendranathe/Web-Scraping/blob/main/Nixon_scrape_Yelp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium

In [42]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re
import time
import openpyxl

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
d = webdriver.Chrome(options=chrome_options)

i = 0
all_reviews = []

while True:
    url = "https://www.yelp.com/biz/richard-nixon-presidential-library-and-museum-yorba-linda-2?start=" + str(i)
    i = i + 10
    d.get(url)
    soup = BeautifulSoup(d.page_source, 'html.parser')
    temp = soup.find('div', {'class': 'css-1qn0b6x', 'id': 'reviews'})
    temp = temp.find('ul', class_ = 'list__09f24__ynIEd')
    reviews = temp.find_all('li', class_= 'css-1q2nwpv') if temp else []
    if len(reviews) < 2:
        break

    for review in reviews:
        userdetails = review.find('div', class_ = 'user-passport-info')

        temp = userdetails.find('a', class_ = 'css-19v1rkv')
        name = temp.text if temp else "Not found"

        temp = userdetails.find('div', class_='elite-badge__09f24__dykWK')
        temp = temp.find('span', class_='css-1adhs7a') if temp else None
        elite = temp.text if temp else "Not found"

        temp = userdetails.find('span', class_ = 'css-qgunke')
        location = temp.text if temp else "Not found"

        userstats =  review.find('div', {'class':'user-passport-stats__09f24__NQxB4'})

        temp = userstats.find('div', {'aria-label':'Friends'})
        friends = temp.text if temp else "Not found"

        temp = userstats.find('div', {'aria-label':'Reviews'})
        review_count = temp.text if temp else "Not found"

        temp = userstats.find('div', {'aria-label':'Photos'})
        pictures = temp.text if temp else "Not found"

        ratings = review.find('div', class_='five-stars__09f24__mBKym')
        ratings_text = ratings['aria-label'] if ratings and 'aria-label' in ratings.attrs else "Not found"

        temp = review.find('span', class_ = 'css-chan6m')
        review_date = temp.text if temp else "Not found"

        temp = review.find('span', class_='reviewQualifier__09f24__iY9mB')
        is_review_updated = True if temp else False

        extra = review.find('div', class_='margin-b2__09f24__CEMjT')

        temp = extra.find('a',class_='css-19v1rkv') if extra else None
        photos = temp.text if temp else "Not found"

        temp = extra.find('span',class_='display--inline__09f24__c6N_k') if extra else None
        check_ins = temp.text if temp else "Not found"

        temp = review.find('p', class_='comment__09f24__D0cxf')
        comment = temp.text if temp else "Not found"

        temp = review.find('div', class_ = 'arrange__09f24__LDfbs vertical-align-middle__09f24__zU9sE css-1qn0b6x')
        temp = temp.find_all('span', class_ = 'css-1xfc281')

        reactions = []
        for tt in temp:
          reactions.append(tt.text)

        # Store the data in a list of lists
        review_data = [name, elite, location, friends, review_count, pictures, ratings_text, review_date, is_review_updated, photos, check_ins, comment, str(reactions)]
        all_reviews.append(review_data)

d.quit()

In [49]:
# Save data to a text file
with open("yelp_reviews.txt", "w", encoding="utf-8") as txt_file:
    for review_data in all_reviews:
        txt_file.write("\t".join(map(str, review_data)) + "\n")

# Save data to an Excel file
wb = openpyxl.Workbook()
ws = wb.active

# Add headers
headers = ["name", "elite", "location", "friends", "review_count", "pictures", "ratings_text", "review_date", "is_review_updated", "photos", "check_ins", "comment", "reactions"]
ws.append(headers)

# Add data to the Excel worksheet
for review_data in all_reviews:
    ws.append(review_data)

# Save the Excel workbook
wb.save("yelp_reviews.xlsx")

# Close the workbook
wb.close()
